In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from autodm.llm import get_llm
from autodm.roll import Dice, DiceType
from autodm.character import Character
from autodm.locaitons import Location, LocationStore, setup_new_locations
from autodm.adventurelog import AdventureLog
from autodm.storyline import StoryLineWithStoryItems

from rich import print

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional, List, Any, Union, Dict
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
character = Character.generate(level=0)
print(character)

In [ ]:
locations = setup_new_locations()
print(locations)

In [ ]:
city, region = locations.current, locations.get(locations.current.parent_name)

In [ ]:
story = StoryLineWithStoryItems.generate(character, city, region, locations)

In [ ]:
print(story)

In [ ]:
building = city.generate_child(type='building', storyline="The adventurers enter the wizard's guild")
room1 = building.generate_child(storyline="The adventurers find and enter the wizard's library.")
room2 = building.generate_child(storyline="The adventurers find and enter thewizard's alchemy lab")
room3 = building.generate_child(storyline="The adventurers find and enter thewizard's training range")

In [ ]:
import networkx as nx
from autodm.locaitons import LOCATION_TYPES

In [ ]:
class LocationGraph:
    def __init__(self):
        self.graph = nx.Graph()

    def set_current_location(self, location_name: str):
        self.current_location = self[location_name]

    def add_location(self, location: Location):
        self.graph.add_node(location.name, **location.model_dump())
        if location.parent_name:
            self.graph.add_edge(location.parent_name, location.name)

    def _node_to_location(self, node: Dict[str, Any]) -> Location:
        return Location(**node)
    
    def _nodes_to_locations(self, nodes: List[Dict[str, Any]]) -> List[Location]:
        return [self._node_to_location(node) for node in nodes]

    def get_children(self, location_name: str):
        nodes = list(self.graph.successors(location_name))
        return self._nodes_to_locations(nodes)

    def get_location(self, location_name: str) -> Optional[Dict[str, str]]:
        if location_name in self.graph:
            node = self.graph.nodes[location_name]
            return self._node_to_location(node)
        return None

    def get_path(self, start_location: str, end_location: str):
        try:
            path = nx.shortest_path(self.graph, start_location, end_location)
            return path
        except nx.NetworkXNoPath:
            return None
        
    def __getitem__(self, name):
        return self._node_to_location(self.graph.nodes[name])
    
    def travel_plan(self, location_name: str) -> Union[str, List[Location]]:
        if location_name not in self.graph:
            return "The location name is unknown. Try creating the location first, then try again."
        paths = self.get_path(self.current_location.name, location_name)
        return f"To reach {location_name} from {paths[0]}, you must travel through the following locations: {','.join(paths[1:-1])}"
        
    def visualize(self):
        plt.figure(figsize=(10, 8))
        pos = nx.spring_layout(self.graph)
        nx.draw(self.graph, pos, with_labels=True, node_size=3000, node_color='skyblue', font_size=10, font_weight='bold')
        node_labels = {node: "\n\n" + data['type'] for node, data in self.graph.nodes(data=True)}
        nx.draw_networkx_labels(self.graph, pos, labels=node_labels, font_size=8, font_color='red')
        plt.title('Location Graph')
        plt.show()

In [ ]:
locations=LocationGraph()
locations.add_location(region)
locations.add_location(city)
locations.add_location(building)
locations.add_location(room1)
locations.add_location(room2)
locations.add_location(room3)
locations.set_current_location(room1.name)

In [ ]:
locations.visualize()

In [ ]:
locations.get_path(room3.name, room2.name)

In [ ]:
locations.graph